# Intensity distribution

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import os 
import cloudpickle

import sympy as sp

from polarimetry.io import (
    mute_jax_warnings,
    perform_cached_doit,
)
from polarimetry.lhcb import load_model
from polarimetry.lhcb.particle import load_particles

mute_jax_warnings()
particles = load_particles("../data/particle-definitions.yaml")
model = load_model("../data/model-definitions.yaml", particles, model_id=0)

In [ ]:
unfolded_intensity_expr = perform_cached_doit(model.full_expression)

## Definition of free parameters

In [ ]:
free_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if isinstance(symbol, sp.Indexed)
    if "production" in str(symbol)
}
fixed_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if symbol not in free_parameters
}
subs_intensity_expr = unfolded_intensity_expr.xreplace(fixed_parameters)

In [ ]:
def sigma3via12() :
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m1, m2, m3 = sp.symbols("m1:4", nonnegative=True)
    m0 = sp.Symbol("m0", nonnegative=True)

    return {s3: m1**2+m2**2+m3**2+m0**2-s1-s2}

In [ ]:
dict_forms = {
    "intensity" : unfolded_intensity_expr,
    "variables" : {k: v.doit() for (k, v) in model.variables.items()},
    "parameter_defaults" : model.parameter_defaults,
    "sigma3" : sigma3via12(),
    "Hproduction" : free_parameters,
}

In [ ]:
docs_dir = os.getcwd()
filename = f"{docs_dir}/.sympy-cache-default-model.pkl"

In [ ]:
with open(filename, "wb") as f:
    cloudpickle.dump(dict_forms, f)